# Fait par:
    -DIGUIRI Gochede
    -NGANGOM Arnella

    ALPHA POSE TRAINING ON COCO
    
    Attendre une dizaine de secondes après avoir lancé le main pour que l'entrainement commence
    Pour modifier les principaux paramètres d'entrainement il faut modifier le fichier opt.py 
    

clonage du github perso modifié d'alphapose
telechargement du modèle de base

---



In [1]:
!git clone https://github.com/diguiri/train_alphapose.git
!gdown --id 1OPORTWB2cwd5YTVBX-NE8fsauZJWsrtW 
!cp /content/duc_se.pth /content/train_alphapose

Cloning into 'train_alphapose'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 52 (delta 5), reused 43 (delta 1), pack-reused 0
Unpacking objects: 100% (52/52), done.
Downloading...
From: https://drive.google.com/uc?id=1OPORTWB2cwd5YTVBX-NE8fsauZJWsrtW
To: /content/duc_se.pth
239MB [00:01, 131MB/s]


telechargment du dataset coco_lite avec le fichier h5 qui permet au modèle d'associer les images et keypoints de manière optimisée

In [2]:
!gdown --id 1n7U4VRwS9DlJ4CfNczi3pt5XrmYsPCG4
!unzip coco.zip -d /content/train_alphapose/

Downloading...
From: https://drive.google.com/uc?id=1n7U4VRwS9DlJ4CfNczi3pt5XrmYsPCG4
To: /content/coco.zip
356MB [00:02, 122MB/s]
Archive:  coco.zip
  inflating: /content/train_alphapose/coco/annot_coco.h5  
   creating: /content/train_alphapose/coco/images/
  inflating: /content/train_alphapose/coco/images/000000000139.jpg  
  inflating: /content/train_alphapose/coco/images/000000000872.jpg  
  inflating: /content/train_alphapose/coco/images/000000000885.jpg  
  inflating: /content/train_alphapose/coco/images/000000001268.jpg  
  inflating: /content/train_alphapose/coco/images/000000001296.jpg  
  inflating: /content/train_alphapose/coco/images/000000001353.jpg  
  inflating: /content/train_alphapose/coco/images/000000001584.jpg  
  inflating: /content/train_alphapose/coco/images/000000001761.jpg  
  inflating: /content/train_alphapose/coco/images/000000002006.jpg  
  inflating: /content/train_alphapose/coco/images/000000002261.jpg  
  inflating: /content/train_alphapose/coco/images/

In [3]:
!pip install tensorboardX #installation de la librairie tensorboardX

     |████████████████████████████████| 317kB 8.4MB/s 


navigation dans le repertoire de travail

In [1]:
%cd /content/train_alphapose 

/content/train_alphapose


In [2]:
#Import des librairies necessaires

import torch
import torch.utils.data
from utils.dataset import coco
from opt import opt
from tqdm import tqdm

from utils.eval import DataLogger, accuracy
from utils.img import flip, shuffleLR
from evaluation import prediction
import torch.nn as nn
import torch
from models.layers.DUC import DUC
from models.layers.SE_Resnet import SEResnet
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

from torchvision import datasets, transforms
from tensorboardX import SummaryWriter
import os

Definition du modèle pytorch

In [3]:
def createModel():
    return FastPose_SE() #modèle fastPose basé sur un resnet101 avec une architecture propriétaire


class FastPose_SE(nn.Module):
    conv_dim = 128

    def __init__(self):
        super(FastPose_SE, self).__init__() 
        #définition de couches supplémentaires. 
        self.preact = SEResnet('resnet101')

        self.suffle1 = nn.PixelShuffle(2)
        self.duc1 = DUC(512, 1024, upscale_factor=2)
        self.duc2 = DUC(256, 512, upscale_factor=2)

        self.conv_out = nn.Conv2d(
            self.conv_dim, 33, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        out = self.preact(x)
        out = self.suffle1(out)
        out = self.duc1(out)
        out = self.duc2(out)

        out = self.conv_out(out)
        return out
    

Definition de la fonction d'entrainement, necessité de trancher la sortie en prenant uniquement les 17 premiers tenseurs car le model de base a été entrainé sur 33 keypoints en même temps qui correspondent au 17 keypoints de coco + les 16 keypoints du MPII

In [4]:
    
def train(train_loader, m, criterion, optimizer, writer):
  lossLogger = DataLogger()
  accLogger = DataLogger()
  m.train() # Activer le mode entrainement  du modèle

  train_loader_desc = tqdm(train_loader)

  for i, (inps, labels, setMask, imgset) in enumerate(train_loader_desc):
        inps = inps.to(device).requires_grad_()
        labels = labels.to(device)
        setMask = setMask.to(device)
        out = m(inps).narrow(1,0,17) #activation du réseau puis découpe de la de sortie 
                                     #afin de considerer uniquement les 17 tenseurs correspondant aux keypoints de coco
       

        loss = criterion(out.mul(setMask), labels) ## calcul de l'erreur de sortie avec le criterion qui est une classe propriétaire

        acc = accuracy(out.data.mul(setMask), labels.data, train_loader.dataset) # calcul précision

        accLogger.update(acc[0], inps.size(0)) #mise a jour 
        lossLogger.update(loss.item(), inps.size(0))

        optimizer.zero_grad()#initialisation des gradients
        loss.backward()#rétro-propagation du gradient
        optimizer.step()#adaptation des poids des neurones

        opt.trainIters += 1
        # Tensorboard
        writer.add_scalar(
            'Train/Loss', lossLogger.avg, opt.trainIters)
        writer.add_scalar(
            'Train/Acc', accLogger.avg, opt.trainIters)

        # TQDM
        train_loader_desc.set_description(
            'loss: {loss:.8f} | acc: {acc:.2f}'.format(
                loss=lossLogger.avg,
                acc=accLogger.avg * 100)
        )

  train_loader_desc.close()

  return lossLogger.avg, accLogger.avg

Fonction de validation avec découpe de la sortie

In [5]:
def valid(val_loader, m, criterion, optimizer, writer):
    lossLogger = DataLogger()
    accLogger = DataLogger()
    m.eval()

    val_loader_desc = tqdm(val_loader)

    for i, (inps, labels, setMask, imgset) in enumerate(val_loader_desc):
        inps = inps.to(device)
        labels = labels.to(device)
        setMask = setMask.to(device)

        with torch.no_grad():
            out = m(inps).narrow(1,0,17) #découpe du tenseur de sortie afin de considerer uniquement les 17 tenseurs de coco

            loss = criterion(out.mul(setMask), labels)

            flip_out = m(flip(inps))
            flip_out = flip(shuffleLR(flip_out, val_loader.dataset))

            out = (flip_out.narrow(1,0,17) + out) / 2 #découpe du tenseur de sortie afin de considerer uniquement les 17 tenseurs de coco
        out = out.narrow(1,0,17) #découpe du tenseur de sortie afin de considerer uniquement les 17 tenseurs de coco
        acc = accuracy(out.mul(setMask), labels, val_loader.dataset)

        lossLogger.update(loss.item(), inps.size(0))
        accLogger.update(acc[0], inps.size(0))

        opt.valIters += 1

        # Tensorboard
        writer.add_scalar(
            'Valid/Loss', lossLogger.avg, opt.valIters)
        writer.add_scalar(
            'Valid/Acc', accLogger.avg, opt.valIters)

        val_loader_desc.set_description(
            'loss: {loss:.8f} | acc: {acc:.2f}'.format(
                loss=lossLogger.avg,
                acc=accLogger.avg * 100)
        )

    val_loader_desc.close()

    return lossLogger.avg, accLogger.avg

Definition de la fonction principale, les principaux parametres d'entrainemnet se trouvent dans le fichier opt.py 

In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu') # choix du périphérique d'entrainenemt

m = createModel().to(device) #Création du modèle
m.load_state_dict(torch.load('duc_se.pth')) #Chargement du checkpoint de base

criterion = torch.nn.MSELoss().to(device)


optimizer = torch.optim.RMSprop(m.parameters(), #choix optimiseur les paramètres se trouvent dans la classe opt.py
                                    lr=opt.LR,
                                    momentum=opt.momentum,
                                    weight_decay=opt.weightDecay)

#initialisation du writer 
writer = SummaryWriter(
    '.tensorboard/{}/{}'.format(opt.dataset, opt.expID))

#préparation du dataset

train_dataset = coco.Mscoco(train=True)
val_dataset = coco.Mscoco(train=False)

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=opt.trainBatch, shuffle=True, num_workers=opt.nThreads, pin_memory=True)

val_loader = torch.utils.data.DataLoader(
    val_dataset, batch_size=opt.validBatch, shuffle=False, num_workers=opt.nThreads, pin_memory=True)

# Transfert du modèle 
m = torch.nn.DataParallel(m).to(device)

#Début d'entrainement
for i in range(opt.nEpochs):
    opt.epoch = i

    print('############# Starting Epoch {} #############'.format(opt.epoch))
    loss, acc = train(train_loader, m, criterion, optimizer, writer) #Entrainenent

    print('Train-{idx:d} epoch | loss:{loss:.8f} | acc:{acc:.4f}'.format(
        idx=opt.epoch,
        loss=loss,
        acc=acc
    ))

    opt.acc = acc
    opt.loss = loss
    m_dev = m.module
    if i % opt.snapshot == 0: #sauvegarde du modèle tout les "opt.snaptshot" epoch par défaut "opt.snaptshot = 1"
        torch.save(
            m_dev.state_dict(), "save.pkl")
        torch.save(
            opt, 'option.pkl')
        torch.save(
            optimizer, 'optimizer.pkl')

    loss, acc = valid(val_loader, m, criterion, optimizer, writer) #validataion

    print('Valid-{idx:d} epoch | loss:{loss:.8f} | acc:{acc:.4f}'.format(
        idx=i,
        loss=loss,
        acc=acc
    ))


writer.close()




  0%|          | 0/199 [00:00<?, ?it/s]

############# Starting Epoch 0 #############


loss: 0.00140076 | acc: 0.98: 100%|██████████| 199/199 [02:36<00:00,  1.27it/s]


Train-0 epoch | loss:0.00140076 | acc:0.0098


loss: 0.00105697 | acc: 2.60:  30%|██▉       | 1175/3968 [01:17<03:12, 14.48it/s]

KeyboardInterrupt: ignored